In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import style
import seaborn as sns
from itertools import combinations
from decimal import Decimal
import statsmodels.sandbox.stats.multicomp

Make fake sample data. In this case 10,000 "frames" with corresponding random numbers between 1 and 20

In [ ]:
np.random.seed(seed=5)
x = np.random.randint(1,21, size=(1, 10000)).tolist()[0]
# print(x)
# print(max(x))
y = list(range(1,10001))
# print(y)

df = pd.DataFrame(
    {'frame': y,
     'df_f': x,
    })

Apply a sliding window average to smooth out some noise

In [ ]:
df['df_f_100'] = df['df_f'].rolling(window=100, center=True).mean()
# print(df.head(10))

Define function to get a range of frames around a given frame and variable values

In [ ]:
frame_value = np.random.randint(300,900, size=(1,10)).tolist()[0]
frame_length = 20

def subset_frame(data, frame_value, frame_length):
    return data[(data['frame'] >= (frame_value - frame_length)) & (data['frame'] < (frame_value+frame_length))]

Make dataframe for making a raster heatmap

In [ ]:
count = 1

for i in range(len(frame_value)):
    curr_fv = frame_value[i]
    tmp = subset_frame(df,curr_fv,frame_length)
    tmp['trial'] = count
    count += 1
    tmp['new_frame'] = range(1,(len(tmp)+1))
    if i == 0:
        df2 = tmp
    else:
        df2 = pd.concat([df2, tmp])
        
# print(df2)

recast from long to wide

In [ ]:
df2 = df2.pivot(index='trial', columns='new_frame', values='df_f_100')
# print(df2.head())

In [ ]:
df2[df2.columns[(frame_length):((frame_length)*2)]] += 2

In [ ]:
fig_size = [10,6]
plt.rcParams["figure.figsize"] = fig_size

sns.heatmap(df2, yticklabels=False, xticklabels=False, cmap='inferno')

Make data frame for plotting line chart

In [ ]:
count = 1

for i in range(len(frame_value)):
    curr_fv = frame_value[i]
    tmp = subset_frame(df,curr_fv,frame_length)
    tmp['new_frame'] = range((len(tmp)))
    tmp = tmp[['df_f_100', 'new_frame']].copy()
    tmp.rename(columns={'df_f_100': count}, inplace=True)
    count += 1
    if i == 0:
        df3 = tmp
    else:
        df3 = df3.merge(tmp, left_on='new_frame', right_on='new_frame', how='left')

df3.set_index('new_frame', inplace=True)

# print(df3.head(3))

Change your fake data to different fake data. Make the "post event" window have higher values than the "pre event" window so it looks like something happened at the event frame.

In [ ]:
df3.iloc[21:25] += 1
df3.iloc[25:30] += 2
df3.iloc[30:41] += 1
# print(df3.head())

Make numpy arrays for graphing

In [ ]:
y = np.array(df3.mean(axis=1))
y = np.round(y, 2)
x = np.array(df3.index.values)
e = np.array([df3.std(axis=1)[i]/np.sqrt(df3.shape[1]) for i in range(len(df3.std(axis=1)))])

Define functions for making combinations and differences in means and p values, for a permutation test

In [ ]:
def make_combinations(v1, v2):
    alls = np.concatenate((v1,v2))
    N = len(alls)
    n = len(v1)
    M = np.array(range(N))
    q = np.array(list(combinations(M,n)))
    q = q[np.random.choice(q.shape[0], 1000, replace=False),:]
    return q

def make_dmeans(v1, v2, q):
    alls = np.concatenate((v1,v2))
    dmeans = np.empty(shape=[1,1000])
    for i in range(len(q)):
        dmeans[0,i] = np.mean(alls[q[i]]) - np.mean(alls[q[-i]])
    dmeans = dmeans.tolist()[0]
    return dmeans

def p_value(true_mean_dif, dec_num):
    pVal = (np.sum(dmeans <= -abs(true_mean_dif)) + np.sum(dmeans >= abs(true_mean_dif)))/1000
    pVal = round((Decimal(pVal)),dec_num)
    return pVal

For each trial, average frames 1-15. This is the baseline vector population for comparing all other time points

In [ ]:
v1 = np.array(df3.loc[1:15].mean(axis=0))
print(v1)
print(len(v1))

v2 = np.array(df3.loc[30])
print(v2)
print(len(v2))

In [ ]:
A = np.empty(shape=[(frame_length*2)-15])
print(A)

In [ ]:
A = np.empty(shape=[(frame_length*2)-15])
v1 = np.array(df3.loc[1:15].mean(axis=0))
count = 0

for i in range(15,40):   
    np.random.seed(seed=5)
    v2 = np.array(df3.loc[i])
    q = make_combinations(v1,v2)
    dmeans = make_dmeans(v1, v2, q)
    d0 = np.mean(v1) - np.mean(v2)
    A[count] = p_value(d0, 5)
    count += 1
print(A)

correct pvals for multiple comparisons. Benjamini Hochberg

In [ ]:
A = statsmodels.sandbox.stats.multicomp.multipletests(A, alpha=0.05, method='fdr_bh', returnsorted=False)
A = A[1]
print(A)

In [ ]:
D = np.copy(y)
print(D)

In [ ]:
# print(len(df3))
# (len(df3) - len(A))

B = np.empty(shape=[(len(df3) - len(A))])
B[B == 0] = 'nan'
print(B)

In [ ]:
C = np.concatenate((B,A), axis=0)
print(C)

In [ ]:
for i in range(len(C)):
    if C[i] > 0.05:
        D[i] = 'nan'
    elif C[i] <= 0.05:
        D[i] = D[i]
    else:
        D[i] = 'nan'
        
print(D)

In [ ]:
x_frames = np.array(range(40))
# print(x_frames)
# print(max(x_frames))

Make graph with all your data. The dark black line is the mean of all the traces, the light gray lines are the individual traces, the blue is the shaded standard error, and the overlaid red dots are all of the "frames" that are significantly different relative to the baseline vector.

In [ ]:
fig_size = [10,7]
plt.rcParams["figure.figsize"] = fig_size

sns.set_style("whitegrid", {'axes.grid' : False, 'axes.linewidth': 1.5, 'axes.edgecolor': '0', 'axes.labelcolor': '0'})
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15)

plt.plot(x, y, 'k-', linewidth=2.0)
plt.fill_between(x, y-e, y+e, color='b', alpha=0.5)
plt.xlabel('"Frames"', size=15)
plt.axvline(x=(frame_length), linewidth=2, color='teal', ls='--')
# plt.xticks(np.arange(min(x), max(x)+1, 5.0))
plt.yticks(np.arange(min(y), max(y)+1, 1))
plt.ylabel('AU', size=15)
sns.despine()
plt.plot(df3, "grey", linewidth=0.5, alpha=0.8)
plt.xlim([min(x),max(x+1)])
plt.plot(x_frames, D, 'ro', markersize = 10)

plt.show()